# Train our Philosophical Document Embedding Model

In [1]:
import torch

import loader
import models
import utility


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

c:\Users\lucas\Desktop\UCSD\256\philosophical_oracle\philosophy_oracle\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [ ]:
filename_category_mapping, category_label_mapping = loader.load_labeling_mappings()
filename_label_mapping = filename_category_mapping
for key, value in filename_label_mapping.items():
    filename_label_mapping[key] = int(category_label_mapping[value])

loader_params = {
    "num_labels": len(category_label_mapping),
    "batch_size": 4,
    "chunk_size": 1000,
    "chunk_overlap": 10,
    "balance_multiplier": 2,
}

train_folders = ["base_texts", "philosophize_this_transcripts", "wikipedia"]
test_folders = ["gpt_summaries"]
filename_category_mapping, category_label_mapping = loader.load_labeling_mappings()

train_dataloader = loader.get_dataloader(train_folders, loader_params, filename_label_mapping, balance_data=True, print_info=False)
test_dataloader = loader.get_dataloader(test_folders, loader_params, filename_label_mapping, balance_data=False, print_info=False)

AttributeError: module 'os' has no attribute 'join'

In [ ]:
model_params = {
    "input_dim": 384,
    "hidden_dim": 128,
    "output_dim": loader_params['num_labels'],
    "dropout": 0.35
}
train_params = {
    "epochs": 10,
    "learning_rate": 1e-3
}



model = models.Classifier(**model_params)
optim = torch.optim.Adam(model.parameters(), lr=train_params['learning_rate'])
model.to(device)

print(f"Epochs  || Train Loss | Train Acc || Test Loss | Test Acc")
for epoch in range(train_params['epochs']):
    train_loss = []
    num_correct, num_total = 0, 0
    for iter, (embeddings, labels) in enumerate(train_dataloader):
        embeddings, labels = embeddings.to(device), labels.to(device)        
        optim.zero_grad()
        logits = model(embeddings)
        
        loss, correct_preds = utility.compute_loss_preds(logits, labels)
        loss.backward()
        optim.step()

        num_correct += correct_preds
        num_total += labels.shape[0]
        train_loss.append(loss.cpu().item())

    eval_loss, eval_acc = utility.evaluate_model(model, test_dataloader, device)
    train_loss = torch.tensor(train_loss)
    print(f" {epoch+1:<2}/{train_params['epochs']:>3} ||    {train_loss.mean():.3f}   |   {num_correct/num_total:.3f}   ||   {eval_loss:.3f}   |  {eval_acc:.3f}  ")

Epochs  || Train Loss | Train Acc || Test Loss | Test Acc


ValueError: too many dimensions 'str'